In [ ]:
import cv2
import numpy as np
import keyboard
import traceback

# Function to perform keyboard actions based on the position of the red object
def control_with_red_object(frame, mask):
    # Find contours of the red object
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Get the largest contour (assumed to be the red object)
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        center_x = x + w // 2
        frame_height, frame_width, _ = frame.shape
        
        # Draw a rectangle around the detected red object
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        # Calculate the relative position of the object from the center
        offset_from_center = center_x - (frame_width // 2)
        sensitivity = abs(offset_from_center) / (frame_width // 2)  # Sensitivity scales with distance

        # Debugging: Show the center X value and offset from the center on the frame
        cv2.putText(frame, f"Center X: {center_x}", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, f"Offset: {offset_from_center}", (40, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Define a smaller neutral region (5% of screen width)
        neutral_threshold = frame_width * 0.05  # 5% of the screen width
        
        # Control logic based on the position and sensitivity of the object
        if center_x < (frame_width // 2) - neutral_threshold:  # Left side, excluding small neutral region
            keyboard.press("left arrow")
            keyboard.release("right arrow")
            cv2.putText(frame, f"LEFT (Sensitivity: {sensitivity:.2f})", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        elif center_x > (frame_width // 2) + neutral_threshold:  # Right side, excluding small neutral region
            keyboard.press("right arrow")
            keyboard.release("left arrow")
            cv2.putText(frame, f"RIGHT (Sensitivity: {sensitivity:.2f})", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        else:  # Small neutral region in the middle of the screen
            keyboard.release("left arrow")
            keyboard.release("right arrow")
            cv2.putText(frame, "NEUTRAL", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
        
    else:
        # If no red object is detected, do nothing (do not press any keys)
        keyboard.release("left arrow")
        keyboard.release("right arrow")
        cv2.putText(frame, "NO OBJECT DETECTED", (40, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

# Instructions: Open the desired racing game app before running this program.
# Set the focus to the app by clicking on its window after executing the code.

try:
    # Getting video input from the camera
    capture = cv2.VideoCapture(0)

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            continue
        
        frame = cv2.flip(frame, 1)

        # Convert the frame to HSV for color detection
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define the range of red color in HSV (two ranges needed for red)
        lower_red1 = np.array([0, 70, 50])    # Lower bound for red hue
        upper_red1 = np.array([10, 255, 255]) # Upper bound for red hue

        lower_red2 = np.array([170, 70, 50])  # Another lower bound for red hue
        upper_red2 = np.array([180, 255, 255])# Another upper bound for red hue

        # Create masks for red color (handling wrap-around of red hue at 180 degrees)
        mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)

        # Combine both masks
        mask = cv2.bitwise_or(mask1, mask2)

        # Perform actions based on the position of the red object
        control_with_red_object(frame, mask)

        # Display the mask and original frame
        cv2.imshow('Mask', mask)
        cv2.imshow('Racing Game Controller', frame)
        
        if cv2.waitKey(1) == 27:  # Escape (ESC) key
            break
            
    cv2.destroyAllWindows()
    capture.release()

except Exception as e:
    cv2.destroyAllWindows()
    capture.release()
    traceback.print_exc()


KeyboardInterrupt: 